# Practica APA - predicció superfície cremada d'un incendi
Sergi Curto Panisello,
Joan Melchor Lladó

## Imports

In [50]:
import pandas as pd
import numpy as np
import tabulate

## Obtenim les dades
Procedim a agafar les dades en format csv descarregades de https://datos.civio.es/dataset/todos-los-incendios-forestales/

In [39]:
df = pd.read_csv('fires-all.csv')
print(df.describe())

                 id    superficie           lat           lng  \
count  8.264000e+04  82640.000000  82616.000000  82616.000000   
mean   2.007259e+09     19.888085     41.763721     -5.664360   
std    4.247106e+06    223.787536      2.019672      4.394031   
min    2.001010e+09      1.000000      0.490720  -1000.000000   
25%    2.003390e+09      1.500000     40.876544     -7.257298   
50%    2.006330e+09      3.000000     42.371123     -6.019478   
75%    2.011150e+09      7.150000     43.143145     -4.303078   
max    2.015510e+09  28879.100000     87.824157    242.755603   

       latlng_explicit   idcomunidad   idprovincia   idmunicipio  \
count     82640.000000  82640.000000  82640.000000  82640.000000   
mean          0.741626      7.850278     28.411375     77.795946   
std           0.437743      5.419922     11.727155     98.867769   
min           0.000000      1.000000      1.000000      1.000000   
25%           0.000000      3.000000     21.000000     27.000000   
50%   

In [51]:
# Si separem solament Catalunya
cat = df.loc[df['idcomunidad'] == 2]
print(cat.head)
print('Shape')
print(cat.shape)

print("ID especial")
print(pd.concat([df.loc[df['id'] == 2001320802], df.loc[df['id'] == 2002151000]]).to_markdown())
# print(df.loc[df['id'] == 2002151000])

print("ID cremat molt")
print(df.loc[df['superficie'] == 28879.10])

<bound method NDFrame.head of                id  superficie       fecha        lat       lng  \
423    2001080017        1.00  2001-02-22  41.667672  2.066639   
424    2001080032        6.00  2001-03-29  41.300741  1.666971   
425    2001080039        1.00  2001-04-07  41.474125  1.973610   
426    2001080041        1.33  2001-04-08  41.483501  1.584177   
427    2001080043        2.00  2001-04-08  41.511833  2.321732   
...           ...         ...         ...        ...       ...   
82298  2015430102        1.63  2015-11-25  41.266186  1.154032   
82299  2015430116       23.23  2015-06-09  41.175501  0.813588   
82300  2015430117       45.03  2015-08-24  40.940658  0.826569   
82301  2015430118       29.00  2015-08-29  41.268525  0.883916   
82302  2015430119        6.76  2015-10-22  41.004475  0.491856   

       latlng_explicit  idcomunidad  idprovincia  idmunicipio  \
423                  1            2            8          223   
424                  1            2            

## Preprocessing

### Tractament de missing values

In [34]:
# Columnes amb missing values, també es pot veure al describe del
# dataset a les variables que no tinguin un count de 82640
print(df.columns[df.isnull().any()].tolist())

# Sobre la "causa supuesta" és normal que hi hagin instàncies sense valor ja que 
# idmunicipio 999, i 998 ens carreguem els que no tinguin lat i long ja que són incendis originats fora d'Espanya
df.dropna(subset=['lat', 'lng'], inplace=True)

# Comencem per emplenar els missing values de muertos i heridos ja que
# si no hi han dades suposarem que són 0.
df['muertos'].fillna(0, inplace=True)
df['heridos'].fillna(0, inplace=True)

# Causa supuesta la posem a 0
df['causa_supuesta'].fillna(0, inplace=True)

# Posem a 0 gastos i perdidas
df['gastos'].fillna(0, inplace=True)
df['perdidas'].fillna(0, inplace=True)

# df.describe()

print(df.columns[df.isnull().any()].tolist())

['lat', 'lng', 'causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']
[]


In [35]:
aux = df.loc[df['idmunicipio'] == 999]
latMissing = df.loc[np.isnan(df['lat'])]
print(aux.shape)
print(latMissing.shape)
print(latMissing)

(72, 21)
(0, 21)
Empty DataFrame
Columns: [id, superficie, fecha, lat, lng, latlng_explicit, idcomunidad, idprovincia, idmunicipio, municipio, causa, causa_supuesta, causa_desc, muertos, heridos, time_ctrl, time_ext, personal, medios, gastos, perdidas]
Index: []

[0 rows x 21 columns]


## Separem target del dataset
Volem predir la superfície cremada donat un incendi per tant separem aquesta variable de la resta.

In [36]:
X = df.drop('superficie', axis=1)
y = df['superficie']

# Comprovem que al target no hi hagin nulls
print(y.isnull().values.any())


False
